In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime
from pyspark.sql.functions  import from_unixtime
from pyspark.sql.functions  import to_date
from pyspark.sql import Row
from pyspark.sql.functions import to_json, struct
from pyspark.sql import functions as F

In [0]:
#Creating the schema for the vehicle data json structure
jsonschema = StructType() \
.add("id", StringType()) \
.add("timestamp", TimestampType()) \
.add("rpm", IntegerType()) \
.add("speed", IntegerType()) \
.add("kms", IntegerType()) 

In [0]:
TOPIC = "kafkaenabledhub"
BOOTSTRAP_SERVERS = "kafkaenabledeventhubns.servicebus.windows.net:9093"
EH_SASL = "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username=\"$ConnectionString\" password=\"Endpoint=sb://kafkaenabledeventhubns.servicebus.windows.net/;SharedAccessKeyName=sendreceivekafka;SharedAccessKey=4vxbVsadasdasdasdp440+SFHpRyQVIpMeXvoVE=\";"
GROUP_ID = "$Default"

# // Read stream using Spark SQL (structured streaming)
# // consider adding .option("startingOffsets", "earliest") to read earliest available offset during testing
kafkaDF = spark.readStream \
    .format("kafka") \
    .option("subscribe", TOPIC) \
    .option("kafka.bootstrap.servers", BOOTSTRAP_SERVERS) \
    .option("kafka.sasl.mechanism", "PLAIN") \
    .option("kafka.security.protocol", "SASL_SSL") \
    .option("kafka.sasl.jaas.config", EH_SASL) \
    .option("kafka.request.timeout.ms", "60000") \
    .option("kafka.session.timeout.ms", "60000") \
    .option("kafka.group.id", "POC") \
    .option("failOnDataLoss", "false") \
    .option("startingOffsets", "latest") \
    .load()




In [0]:
#Converting binary datatype to string for the dataframe columns. Without this you cannot use from_json function as it expects the column datatype as string not binary
kafkaDF=kafkaDF.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
#Adding new column vehiclejson which is a struct and has 5 columns id, timestamp,rpm,speed and kms
newkafkaDF=kafkaDF.withColumn('vehiclejson', from_json(col('value'),schema=jsonschema))

In [0]:
kafkajsonDF=newkafkaDF.select("key","value", "vehiclejson.*")

In [0]:
#you can uncomment and run the below command to view the column values
# display(kafkajsonDF)

key,value,id,timestamp,rpm,speed,kms
null,"{""id"": ""4ba6e242-7282-4367-b09c-7568e4b12a58"", ""timestamp"": ""2021-03-14 02:23:31.016301"", ""rpm"": 9, ""speed"": 85, ""kms"": 822}",4ba6e242-7282-4367-b09c-7568e4b12a58,2021-03-14T02:23:31.016+0000,9,85,822
null,"{""id"": ""7e0d39fd-7251-483c-92d6-7d6bb5cc164e"", ""timestamp"": ""2021-03-14 02:23:31.289347"", ""rpm"": 18, ""speed"": 82, ""kms"": 883}",7e0d39fd-7251-483c-92d6-7d6bb5cc164e,2021-03-14T02:23:31.289+0000,18,82,883
null,"{""id"": ""405a6bb3-ed92-4a85-996c-9d8b01867923"", ""timestamp"": ""2021-03-14 02:23:31.342846"", ""rpm"": 55, ""speed"": 94, ""kms"": 349}",405a6bb3-ed92-4a85-996c-9d8b01867923,2021-03-14T02:23:31.342+0000,55,94,349
null,"{""id"": ""a18df3df-0a16-484b-9a3b-9a2a2ed39b58"", ""timestamp"": ""2021-03-14 02:23:31.391535"", ""rpm"": 6, ""speed"": 80, ""kms"": 168}",a18df3df-0a16-484b-9a3b-9a2a2ed39b58,2021-03-14T02:23:31.391+0000,6,80,168
null,"{""id"": ""a4cde850-318d-4e91-ba08-1f5cd3ddd561"", ""timestamp"": ""2021-03-14 02:23:31.456819"", ""rpm"": 49, ""speed"": 82, ""kms"": 973}",a4cde850-318d-4e91-ba08-1f5cd3ddd561,2021-03-14T02:23:31.456+0000,49,82,973
null,"{""id"": ""4ba6e242-7282-4367-b09c-7568e4b12a58"", ""timestamp"": ""2021-03-14 02:23:31.500804"", ""rpm"": 48, ""speed"": 89, ""kms"": 671}",4ba6e242-7282-4367-b09c-7568e4b12a58,2021-03-14T02:23:31.500+0000,48,89,671
null,"{""id"": ""7e0d39fd-7251-483c-92d6-7d6bb5cc164e"", ""timestamp"": ""2021-03-14 02:23:31.568811"", ""rpm"": 73, ""speed"": 96, ""kms"": 755}",7e0d39fd-7251-483c-92d6-7d6bb5cc164e,2021-03-14T02:23:31.568+0000,73,96,755
null,"{""id"": ""405a6bb3-ed92-4a85-996c-9d8b01867923"", ""timestamp"": ""2021-03-14 02:23:31.625805"", ""rpm"": 29, ""speed"": 80, ""kms"": 383}",405a6bb3-ed92-4a85-996c-9d8b01867923,2021-03-14T02:23:31.625+0000,29,80,383
null,"{""id"": ""a18df3df-0a16-484b-9a3b-9a2a2ed39b58"", ""timestamp"": ""2021-03-14 02:23:31.673803"", ""rpm"": 55, ""speed"": 100, ""kms"": 207}",a18df3df-0a16-484b-9a3b-9a2a2ed39b58,2021-03-14T02:23:31.673+0000,55,100,207
null,"{""id"": ""a4cde850-318d-4e91-ba08-1f5cd3ddd561"", ""timestamp"": ""2021-03-14 02:23:31.716811"", ""rpm"": 25, ""speed"": 97, ""kms"": 531}",a4cde850-318d-4e91-ba08-1f5cd3ddd561,2021-03-14T02:23:31.716+0000,25,97,531


### Default (No Trigger option specified)

In [0]:
#Writing the streaming data to Delta tables with default trigger i.e. no trigger option is specified

kafkajsonDF.selectExpr(
                  "id"	  \
                  ,"timestamp"	   \
                  ,"rpm"	\
                  ,"speed" \
                  ,"kms" ) \
.writeStream.format("delta") \
.outputMode("append") \
.option("checkpointLocation", "dbfs:/Vehiclechkpointkafkaeventhub_Demo_trigger/") \
.option("mergeSchema", "true") \
.start("dbfs:/VehiclechkpointKafkaEventHub_Delta_trigger") 

Out[18]: <pyspark.sql.streaming.StreamingQuery at 0x7f4e04080250>

In [0]:
%sql
-- Creating the table on delta location
CREATE TABLE IF NOT EXISTS VehicleDetails_KafkaEnabledEventHub_Delta_Trigger
USING DELTA
LOCATION "dbfs:/VehiclechkpointKafkaEventHub_Delta_trigger/"

In [0]:
%sql
-- You can keep running this cell continuously  and you will find the numbers growing as data is processed as it arrives.
select count(*) from 
VehicleDetails_KafkaEnabledEventHub_Delta_Trigger

count(1)
400


### Fixed Interval (trigger(processingTime = duration))

In [0]:
kafkajsonDF.selectExpr(
                  "id"	  \
                  ,"timestamp"	   \
                  ,"rpm"	\
                  ,"speed" \
                  ,"kms" ) \
.writeStream.format("delta") \
.outputMode("append") \
.option("checkpointLocation", "dbfs:/Vehiclechkpointkafkaeventhub_Demo_trigger/") \
.option("mergeSchema", "true") \
.trigger(processingTime='30 seconds') \
.start("dbfs:/VehiclechkpointKafkaEventHub_Delta_trigger") 


Out[19]: <pyspark.sql.streaming.StreamingQuery at 0x7f4df705d5d0>

In [0]:
%sql
-- You can keep running this cell continuously and you will find the numbers don't change continuously , it gets updates after every 30 seconds.
select count(*) from 
VehicleDetails_KafkaEnabledEventHub_Delta_Trigger

count(1)
2600


### continuous (trigger(continuous='1 second'))
- Supported Sources
  - Kafka source: All options are supported.
  - Rate source: Good for testing. Only options that are supported in the continuous mode are numPartitions and rowsPerSecond.

- Supported Sinks
  - Kafka sink: All options are supported.
  - Memory sink: Good for debugging.
  - Console sink: Good for debugging. All options are supported. Note that the console will print every checkpoint interval that you have specified in the continuous trigger.

In [0]:

kafkajsonDF.selectExpr(
                  "id"	  \
                  ,"timestamp"	   \
                  ,"rpm"	\
                  ,"speed" \
                  ,"kms" ) \
.writeStream.format("console") \
.outputMode("append") \
.trigger(continuous='1 second') \
.start()


Out[8]: <pyspark.sql.streaming.StreamingQuery at 0x7f961860f390>